In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
def scrollDown(driver, reviewTotal, wait):
    x = len(driver.find_elements_by_xpath("//div[@class='gws-localreviews__general-reviews-block']//div[@class='WMbnJf gws-localreviews__google-review']"))
    lastx = x - 1
    i = 1
    repCount = 0
    while ((x < reviewTotal) and (repCount < 2)):
        pauseScroll(wait)
        elements = driver.find_elements_by_xpath("//div[@class='gws-localreviews__general-reviews-block']//div[@class='WMbnJf gws-localreviews__google-review']")
        current_last = elements[-1]
        current_last.location_once_scrolled_into_view
        lastx = x
        x = len(driver.find_elements_by_xpath("//div[@class='gws-localreviews__general-reviews-block']//div[@class='WMbnJf gws-localreviews__google-review']"))
        if (lastx == x):
            repCount += 1

In [11]:
def getReviewTotal(driver):
    raw = driver.find_element_by_xpath("//span[@class='hqzQac']").text
    count = int(raw.split(" ")[0])
    return count

In [4]:
import numbers
import time
def pauseScroll(wait):
    waitTime = 0
    # if argument is a list
    if (isinstance(wait, list)):
        # check if its length is valid
        if (len(wait) > 2) or (len(wait) < 1):
            raise ValueError
        # if length is valid get random number between vals
        low = wait[0]
        high = wait[1]
        # check for improper types, fix if possible
        if not isinstance(low, numbers.Number):
            if low.isnumeric():
                low = float(low)
            else:
                raise ValueError
        if not isinstance(high, numbers.Number):
            if high.isnumeric():
                high = float(high)
            else:
                raise ValueError
        waitTime = randTime(low, high)
    # if argument is a number
    elif (isinstance(wait, numbers.Number)):
        # get the number
        waitTime = wait
    # if argument is a string
    elif (isinstance(wait, str)):
        # if argument can be parsed as a number
        if (wait.isnumeric()):
            # get time as a number
            waitTime = float(wait)
        else:
            raise ValueError
    time.sleep(waitTime)

In [5]:
from random import random
def randTime(low, high):
    val = random()
    time = low + (val  * (high - low))
    return time

In [6]:
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
# note that this assumes scrolling has already occurred
def scrapeFromList(driver, id=None, name=True, stars=True, text=True, timeSince=True):
    elements = driver.find_elements_by_xpath("//div[@class='gws-localreviews__general-reviews-block']//div[@class='WMbnJf gws-localreviews__google-review']")
    columnsBin = [name, stars, text, timeSince]
    columnsNames = ["name", "stars", "text", "timeSince"]
    columnsPresent = [x for i, x in enumerate(columnsNames) if columnsBin[i]] # gets included columns
    
    # text body
    textList = []
    if (text):
        textPath = ".//div[@style='vertical-align:top']//div[@class='Jtu6Td']//span[@jscontroller='P7L8k']"
        textPathFullExtension = "//span[@class='review-full-text']"
        textList = [0]*len(elements)
        i = 0
        for e in elements:
            try:
                textPartial = e.find_element_by_xpath(textPath+textPathFullExtension)
                e.find_element_by_xpath(textPath+"//span[@role='button']").click()
                textList[i] = e.find_element_by_xpath(textPath).text
            except NoSuchElementException:
                textList[i] = e.find_element_by_xpath(textPath).text
            i += 1
        
    # name
    nameList = []
    if (name):
        namePath = ".//div[@class='TSUbDb']"
        nameList = [e.find_element_by_xpath(namePath).text for e in elements]
    
    # stars
    starsList = []
    if (stars):
        starsPath = ".//div[@style='vertical-align:top']//div[@class='PuaHbe']//g-review-stars[@style='padding-right:7px']//span[@class='Fam1ne EBe2gf']"
        i = 0
        starsList = [0]*len(elements)
        for e in elements:
            raw = e.find_element_by_xpath(starsPath).get_attribute("aria-label")
            starCount = raw.split(" ")[1] # the number of stars
            starsList[i] = float(starCount)
            i += 1
    
    # timeSince
    timeSinceList = []
    if (timeSince):
        timeSincePath = ".//div[@style='vertical-align:top']//div[@class='PuaHbe']"
        timeSinceList = [e.find_element_by_xpath(timeSincePath).text for e in elements]
    
    # put df together
    columnsBin = [name, stars, text, timeSince]
    columnsNames = ["name", "stars", "text", "timeSince"]
    columnsPresent = [x for i, x in enumerate(columnsNames) if columnsBin[i]] # gets included columns
    columns = [nameList, starsList, textList, timeSinceList]
    columns = [c for c in columns if len(c) > 0]
    data = {c:columns[i] for i,c in enumerate(columnsPresent)}
    df = pd.DataFrame(data)
    
    return df

In [23]:
import datetime
import pandas
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
def scrapeFromURLs(urls, combine=True, wait=[2,3]):
    # check that urls are dicts
    if not (isinstance(urls, dict)):
        raise ValueError
        
    # start selenium stuff
    driver = webdriver.Chrome('./chromedriver')
    
    dfs = [0]*len(urls)
    
    # for each id
    i = 0
    for id in urls:
        url = urls.get(id)
        driver.get(url) # go to url
        driver.find_element_by_class_name("hqzQac").click()
        WebDriverWait(driver, 10).until(ec.presence_of_element_located((By.XPATH, "//div[@class='AU64fe']")))
        
        # scroll to bottom
        scrollDown(driver, getReviewTotal(driver), wait)
        
        # scrape from loaded list
        df = scrapeFromList(driver, id)
        
        dfs[i] = df
        i += 1
        
    # combine dfs if necessary
    if (combine):
        df = dfs[0]
        # append all other dfs to the first one, then save to csv
        for i in dfs[1:]:
            df = df.append(i, ignore_index=True)
        df.to_csv("combined_scrape_"+str(datetime.datetime.now()).replace(' ','_').replace(':','_').replace('.','_')+".csv",index=False)
    else:
        for i in dfs:
            i.to_csv("scraped_"+str(datetime.datetime.now()).replace(' ','_').replace(':','_').replace('.','_')+".csv",index=False)
        
    driver.quit()

In [8]:
# TODO
# make scrapeFromURLs take in a filename convention and have csvs save as it
# add check for correct address
# check that reviews button is there, i.e. check whether there is actually a location found

In [9]:
import pandas as pd
def extractURLS(csv, searchTerms, keys, firstIndex, lastIndex):
    baseString = "google.com/search?q="
    
    # read in csv (maybe add value checks later)
    df = pd.read_csv(csv)
    
    #search = df.loc[]
    

SyntaxError: invalid syntax (<ipython-input-9-53274d23d3e7>, line 8)

In [24]:
testURLs = {1:"https://www.google.com/search?q=pizza+hut+142+highway+62+ash+flat+ar", 2:"https://www.google.com/search?q=pizza+hut+620+lincoln+way+ames+ia"}

testdf = scrapeFromURLs(testURLs)
testdf